In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [14]:
pd.read_sql(
    '''
    SELECT * FROM OrderDetails
    ''',
    con,
)

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10
2,3,10248,72,5
3,4,10249,14,9
4,5,10249,51,40
...,...,...,...,...
513,514,10442,11,30
514,515,10442,54,80
515,516,10442,66,60
516,517,10443,11,6


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [7]:
pd.read_sql(
    '''--sql
    SELECT CustomerName, Country, Address
    FROM Customers
    WHERE City = 'Madrid' OR Country IN ('Germany','France')
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [8]:
pd.read_sql(
    '''--sql
    SELECT Country, count(*)
    FROM Customers
    GROUP BY Country
    ORDER BY count(*) DESC
    LIMIT 3
    ''',
    con,
)

,Country,count(*)
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [10]:
pd.read_sql(
    '''--sql
    SELECT ShipperName, OrderDate
    FROM Shippers s
    JOIN Orders o ON s.ShipperID = o.ShipperID
    -- order by cast(to_date(OrderDate, 'DD.MM.YYYY')) -- не работает
    ORDER BY OrderDate desc
    LIMIT 10
    ''',
    con,  parse_dates = {'OrderDate': {'format': '%d.%m.%Y'}}
)

,ShipperName,OrderDate
0,Federal Shipping,1996-12-31
1,Speedy Express,1996-10-31
2,Speedy Express,1996-07-31
3,United Package,1997-01-31
4,Federal Shipping,1996-12-30
5,United Package,1996-10-30
6,Speedy Express,1996-09-30
7,United Package,1996-08-30
8,Federal Shipping,1996-07-30
9,Speedy Express,1997-01-30


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [16]:
pd.read_sql(
    '''--sql
    -- В разработке 
    SELECT 
    o.*
    , p.* 
    FROM Orders o 
    JOIN OrderDetails od ON o.OrderID = od.OrderID 
    JOIN Products p ON od.ProductID = p.ProductID
    ''',
    con,
)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,10248,90,5,04.07.1996,3,11,Queso Cabrales,5,4,1 kg pkg.,21.00
1,10248,90,5,04.07.1996,3,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00
2,10248,90,5,04.07.1996,3,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80
3,10249,81,6,05.07.1996,1,14,Tofu,6,7,40 - 100 g pkgs.,23.25
4,10249,81,6,05.07.1996,1,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00
...,...,...,...,...,...,...,...,...,...,...,...
513,10442,20,3,11.02.1997,2,11,Queso Cabrales,5,4,1 kg pkg.,21.00
514,10442,20,3,11.02.1997,2,54,Tourtière,25,6,16 pies,7.45
515,10442,20,3,11.02.1997,2,66,Louisiana Hot Spiced Okra,2,2,24 - 8 oz jars,17.00
516,10443,66,8,12.02.1997,1,11,Queso Cabrales,5,4,1 kg pkg.,21.00


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

10. Сколько банок крабового мяса всего было заказано.

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)